In [268]:
import pandas as pd
import time
import numpy as np
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [166]:
uh_df = pd.read_csv('../input/unit-history.csv.gz')

In [167]:
start_col = 'Start.Date'
end_col = 'End.Date'
unit_col = 'Unit'
uid_col = 'UID'
lower_bound = '2000-01-01'
upper_bound = ''
cap_date = ''

In [168]:
uh_df.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,NaN,2016-06-05,1,3.0
2,NaN,2007-03-29,2,15.0
3,1971-01-15,1970-06-15,3,22.0
4,1976-08-18,1971-01-16,3,4.0


In [169]:
uh_df.shape

(107786, 4)

In [170]:
uh_df[end_col] = pd.to_datetime(uh_df[end_col])
uh_df[start_col] = pd.to_datetime(uh_df[start_col])

In [171]:
if upper_bound:
    upper_bound = pd.to_datetime(upper_bound)
    assert isinstance(upper_bound, pd.Timestamp), 'upper bound given is not a proper date'
else:
    upper_bound = pd.to_datetime(pd.date_range(start = max(uh_df[start_col]),
                                            periods=2, freq='D'))[1]
uh_df = uh_df[uh_df[start_col] <= upper_bound]
uh_df.loc[(uh_df[end_col].isnull()) |
          (uh_df[end_col] > upper_bound),
          end_col] = upper_bound


In [172]:
uh_df.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,2016-10-10,2016-06-05,1,3.0
2,2016-10-10,2007-03-29,2,15.0
3,1971-01-15,1970-06-15,3,22.0
4,1976-08-18,1971-01-16,3,4.0


In [173]:
if lower_bound:
    uh_df = uh_df[uh_df[end_col] >= lower_bound]

In [174]:
uh_df.shape

(55265, 4)

In [175]:
def expand_times(sd,ed,unit,uid):
    outdf = pd.DataFrame(pd.date_range(start=sd, end=ed, freq='D', name='Date'))
    outdf.insert(1, 'Unit', unit)
    outdf.insert(2, 'UID', uid)
    event = [0] * outdf.shape[0]
    event[0] = 1
    event[-1] = -1
    outdf.insert(3, 'Event', event)
    return outdf

In [176]:
r = uh_df.loc[0]
sd = r[start_col]
ed = r[end_col]
unit = r[unit_col]
uid = r[uid_col]

In [177]:
outdf = {'Dates' : pd.date_range(start=sd, end=pd.to_datetime('2007-04-04'), freq='D', name='Date')}
rows = len(outdf['Dates'])
outdf['Unit'] = unit
outdf['UID'] = uid
event = [0] * rows
event[0] = 1
event[-1] = -1
outdf['Event'] = event

In [178]:
def expand_times2(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return outdf

In [179]:
def expand_times3(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return pd.DataFrame(outdf)

In [180]:
def expand_times4(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return pd.DataFrame.from_dict(outdf)

In [181]:
def ttt():
    tdf = [expand_times2(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()]
    tdf = pd.concat([pd.DataFrame(df) for df in tdf])
    return tdf

In [188]:
%lprun -f expand_times pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [189]:
%lprun -f ttt ttt()

In [185]:
%lprun -f expand_times3 pd.concat([expand_times3(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [186]:
%lprun -f expand_times4 pd.concat([expand_times4(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [187]:
# EXPAND TIMES 3 IS FASTEST

In [197]:
start = time.time()
tdf = pd.concat([expand_times3(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df.iterrows()], ignore)
end = time.time()
print(end - start)

120.43245387077332


In [262]:
# Month test
sd = pd.Period('2005-05-06').asfreq('M').to_timestamp()
ed = pd.Period('2005-05-25').asfreq('M')

In [261]:
sd

Timestamp('2005-05-01 00:00:00')

In [238]:
pd.date_range(start=sd, end=ed, freq='D', name='Date')

DatetimeIndex(['2005-05-06', '2005-05-07', '2005-05-08', '2005-05-09',
               '2005-05-10', '2005-05-11', '2005-05-12', '2005-05-13',
               '2005-05-14', '2005-05-15', '2005-05-16', '2005-05-17',
               '2005-05-18', '2005-05-19', '2005-05-20', '2005-05-21',
               '2005-05-22', '2005-05-23', '2005-05-24', '2005-05-25'],
              dtype='datetime64[ns]', name='Date', freq='D')

In [236]:
pd.date_range(start=sd, freq='MS', name='Date', periods=1)

DatetimeIndex(['2005-06-01'], dtype='datetime64[ns]', name='Date', freq='MS')

In [326]:
def expand_times(sd,ed,unit,uid):
    outdict = {}
    if sd == ed:
        dl = [sd]
        event = 3
        
    else:
        dl = pd.date_range(start=sd, end=ed, freq='MS')
        event = [0] * len(dl)
        event[0] = 1
        event[-1] = 2

    outdict = {'Month' : dl}
    outdict['Event'] = event
    outdict['Unit'] = unit
    outdict['UID'] = uid
    return pd.DataFrame(outdict)

In [327]:
def to_month_start(dates):
    plus_month = np.array([1], dtype='timedelta64[M]')
    minus_day = np.array([1], dtype='timedelta64[D]')
    dates = dates.values.astype('datetime64[M]') + plus_month - minus_day
    return dates

In [321]:
mdf = uh_df[0:1000]

In [322]:
mdf['Start.Date'] = to_month_start(mdf['Start.Date'])
mdf['End.Date'] = to_month_start(mdf['End.Date'])

/Users/invinst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/invinst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [328]:
start = time.time()
%lprun -f expand_times pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in mdf.iterrows()])
end = time.time()
print(end - start)

11.07385778427124


In [305]:
def history_to_panel(hist_df, freq,
                     start_col='Start.Date', end_col='End.Date', unit_col='Unit', uid_col, 'UID'):
    assert freq in ['Day', 'Month', 'Year'], "freq must be 'Day', 'Month', or 'Year'"
    if freq == 'Day':
        
        
    else:

SyntaxError: invalid syntax (<ipython-input-305-6e46eb1c2adb>, line 2)

In [64]:
start = time.time()
full_df = pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[1:1000].iterrows()])
end = time.time()
print(end - start)

2.927866220474243


In [61]:
full_df.head()

,Date,Unit,UID,Event
0,2007-03-29,14.0,1,1
1,2007-03-30,14.0,1,0
2,2007-03-31,14.0,1,0
3,2007-04-01,14.0,1,0
4,2007-04-02,14.0,1,0


In [62]:
full_df.shape

(166137999, 4)

In [63]:
full_df[(full_df['UID']==1) & (full_df['Unit']==3.0)]

,Date,Unit,UID,Event
0,2016-06-05,3.0,1,1
1,2016-06-06,3.0,1,0
2,2016-06-07,3.0,1,0
3,2016-06-08,3.0,1,0
4,2016-06-09,3.0,1,0
5,2016-06-10,3.0,1,0
6,2016-06-11,3.0,1,0
7,2016-06-12,3.0,1,0
8,2016-06-13,3.0,1,0
9,2016-06-14,3.0,1,0
